In [1]:
from imutils.video import VideoStream
import argparse
import imutils
import time
import cv2
import sys
import numpy as np
import os
import utils

In [2]:
def camera_test():
    """used to locate all possible usb cameras on device"""
    all_camera_idx_available = []

    for camera_idx in range(-100, 900):
        cap = cv2.VideoCapture(camera_idx)
        if cap.isOpened():
            print(f'Camera index available: {camera_idx}')
            all_camera_idx_available.append(camera_idx)
            cap.release()
        

In [3]:
camera_test()

Camera index available: 0
Camera index available: 1


In [ ]:
arucoDict = cv2.aruco.DICT_4X4_50
mtx, dist = utils.load_coefficients('calibration_aruco1.yml')

print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)
while True:
    time.sleep(1.0)
    frame = vs.read()

    image = imutils.resize(frame, width = 1000)
    #(corners, ids, rejected) = cv2.aruco.detectMarkers(image, arucoDict, parameters=arucoParams)
    output, rvecs, tvecs= utils.pose_estimation(image, arucoDict, mtx, dist)
    
    #cv2.destroyAllWindows()
    tvec_list, rvec_list = utils.clean_tvec_rvec(tvecs, rvecs)
    if len(tvec_list) != 5:
        print("[ERROR] not all markers were detected, trying again later")
        continue
    width, length = utils.get_width_length(tvec_list)
    possible_points = utils.calculate_possible_points(tvec_list, width, length)
    #print(possible_points)
    prediction, x, y = utils.clean_points_and_vote(width, length, 3, possible_points)
    angle = utils.diff_y_angle(rvec_list)
    prop = .75/width
    print(width)
    print(length)
    print('location ' + str(prop*x) + ' ' + str(prop*y))
    print('predicted box: ' + str(prediction))
    print('angle: ' + str(angle))
    cv2.imshow('test', output)
    cv2.waitKey(0)
    
    
cv2.destroyAllWindows()
vs.stop()

[INFO] starting video stream...
0.21412804657750334
0.2356036476649192
location 0.4195628488651777 0.4706910860897537
predicted box: 8
angle: -123.96197164148386
0.21381541826269598
0.23817722182568768
location 0.43672231929545263 0.5627184989877272
predicted box: 8
angle: -125.16079109457247
0.21694220199154074
0.23703184282501738
location 0.41559462011791054 0.5984655198903986
predicted box: 7
angle: -141.1156036160642
0.21723753450341377
0.23696570519263102
location 0.39572290792366394 0.5607351963238969
predicted box: 7
angle: -102.5600418898492
0.21381541826269598
0.23817722182568768
location 0.24854084755687533 0.5484433178506568
predicted box: 6
angle: 78.93750333575173
0.21694220199154074
0.23703184282501738
location 0.39762163940319584 0.27748359394784833
predicted box: 1
angle: 1.2771583409277205
0.21585885825028361
0.23626264589958873
location 0.39839781669243846 0.27634350988325707
predicted box: 2
angle: -0.7466157619664239
0.2155635257384106
0.23630257135413574
location 0